In [8]:
import gzip
import requests
import random
import numpy as np
import pandas as pd

r/news:

In [2]:
sub = 'news'
size = 500
after = [1615766400, # March 15, 2021 12:00:00 AM GMT
         1611964800, # January 30, 2021 12:00:00 AM GMT
         1626566400, # July 18, 2021 12:00:00 AM GMT
         1609891200, # January 6, 2021 12:00:00 AM GMT
         1616198400, # March 20, 2021 12:00:00 AM GMT
         1637452800, # November 21, 2021 12:00:00 AM GMT
         1634774400, # October 21, 2021 12:00:00 AM GMT
         1640390400, # December 25, 2021 12:00:00 AM GMT
         1627516800, # July 29, 2021 12:00:00 AM GMT
         1628380800, # August 8, 2021 12:00:00 AM GMT
         1626220800, # July 14, 2021 12:00:00 AM GMT
         1610582400, # January 14, 2021 12:00:00 AM GMT
         1613779200, # February 20, 2021 12:00:00 AM GMT
         1618704000, # April 18, 2021 12:00:00 AM GMT
         1636156800 # November 6, 2021 12:00:00 AM GMT
         ]

In [ ]:
rnews = pd.DataFrame(columns=['body'])

for epoch in after:
    url = f'https://api.pushshift.io/reddit/search/comment/?fields=body&subreddit={sub}&size={size}&after={epoch}'
    rnews = pd.concat([rnews, pd.DataFrame.from_dict(requests.get(url).json()['data'])], ignore_index=True)

In [ ]:
rnews

In [ ]:
rnews_notdel = rnews.query('body not in ["[removed]", "[deleted]"]').reset_index().drop(columns=['index'])
rnews_notdel

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='rnews_data.csv')
rnews_notdel.to_csv('rnews_data.zip', index=False, compression=compression_opts)

In [ ]:
rnews = pd.read_csv('rnews_data.csv')
rnews

In [ ]:
random.seed(1)
n = rnews.shape[0]
indices = random.sample(range(n), 30)
rnews.iloc[indices].to_csv('rnews_sample30.txt', index=False)

Amazon book reviews:

In [ ]:
with gzip.open('Kindle_Store_5.json.gz', 'rb') as f:
    reviews_df = pd.read_json(f, orient='records', lines=True)

In [ ]:
random.seed(1)
reviews_data = reviews_df[['reviewText']].iloc[random.sample(range(reviews_df.shape[0]), 1000)]
reviews_data

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='reviews_data.csv')
reviews_data.to_csv('reviews_data.zip', index=False, compression=compression_opts)

In [ ]:
random.seed(1)
reviews_data.reset_index().drop(
        columns=['index']
    ).iloc[random.sample(range(1000), 20)].to_csv(
        'reviews_sample30.csv', index=False
    )

Annotation (AP2):

In [ ]:
import zipfile

with zipfile.ZipFile('data/reviews_data.zip', 'r') as z:
    with z.open('reviews_data.csv') as f:
        reviews_data = pd.read_csv(f)

In [ ]:
reviews_data

In [ ]:
reviews_eval = pd.read_csv('data/reviews_eval_adj.csv')
reviews_exp = pd.read_csv('data/reviews_exp_adj.csv').convert_dtypes()

In [ ]:
display(reviews_eval.head())
display(reviews_exp.head())

In [ ]:
reviews_annotated = pd.concat([reviews_eval, reviews_exp], ignore_index=True)

In [ ]:
reviews_annotated.value_counts('adjudicated')

In [ ]:
adjudicated = reviews_annotated[['text', 'adjudicated']].melt(
        id_vars=['text'], 
        value_vars=['adjudicated'], 
        var_name='adjudicated'
    ).rename(
        columns={'value': 'label'}
    )[['adjudicated', 'label', 'text']]

In [ ]:
adjudicated.to_csv('data/adjudicated.txt', sep='\t', header=False)

In [ ]:
individual_annotations = reviews_eval[['text', 'stalebi', 'zhossainzadeh']].reset_index().melt(
        id_vars=['index', 'text'],
        value_vars=['stalebi', 'zhossainzadeh'],
        var_name='annotator'
    ).sort_values(
        'index', 
        ignore_index=True
    ).rename(
        columns={'value': 'label'}
    )[['index', 'annotator', 'label', 'text']]

In [ ]:
individual_annotations

In [ ]:
individual_annotations.to_csv('data/individual_annotations.txt', sep='\t', index=False, header=False)

AP3:

In [ ]:
ap3_annotated = pd.read_csv('data/3034029249.txt', sep='\t')

In [ ]:
ap3_annotated

In [ ]:
labels_dict = {
    1: 'health',
    2: 'versatility',
    3: 'gourmet experience',
    4: 'convenience',
    5: 'taste'
}

In [ ]:
i = 99
ap3_annotated.loc[i]['text']

In [ ]:
labels_val = [
    2,
    5,
    3,
    5,
    1,
    5,
    1,
    5,
    1,
    5,
    1,
    2,
    5,
    2,
    4,
    5,
    3,
    3,
    5,
    5,
    5,
    1,
    5,
    4,
    1,
    3,
    2,
    5,
    3,
    5,
    2,
    2,
    1,
    5,
    5,
    5,
    5,
    3,
    5,
    2,
    5,
    5,
    3,
    5,
    3,
    1,
    3,
    5,
    2,
    5,
    1,
    3,
    1,
    5,
    3,
    2,
    1,
    1,
    3,
    3,
    1,
    5,
    5,
    5,
    2,
    3,
    3,
    3,
    3,
    1,
    1,
    4,
    3,
    5,
    5,
    1,
    3,
    5,
    4,
    5,
    2,
    2,
    4,
    5,
    1,
    1,
    3,
    5,
    1,
    1,
    3,
    3,
    4,
    3,
    2,
    3,
    1,
    1,
    3,
    3
]

In [ ]:
len(labels_val)

In [ ]:
labels = [labels_dict[i] for i in labels_val]
labels

In [ ]:
ap3_annotated['label'] = pd.Series(labels)
ap3_annotated

In [ ]:
ap3_annotated.to_csv('data/annotations.txt', sep='\t', index=False, header=False)

In [ ]:
ap3_annotated.query('label == "gourmet experience"')

AP4:

In [15]:
bert_labels = pd.read_csv('data/mistakes_bert.csv', index_col=0)

In [16]:
mistakes_bert = bert_labels.query("wrong == 1")

In [126]:
mistakes_bert.query('pred == 2 & TRUE == 1')['text'].to_csv('test.csv')

In [25]:
correct_bert = bert_labels.query("wrong == 0")

In [62]:
bert_labels.query('pred == 1')['text'].to_csv('bert_labels_1.csv')

In [46]:
for i in range(1, 5):
    print('Average length of predicted {}s: {}'.format(i, 
                                                       bert_labels.query('pred == {}'.format(str(i)))['text'].str.split().apply(lambda l: len(l)).mean()))


Average length of predicted 1s: 14.894736842105264
Average length of predicted 2s: 50.12658227848101
Average length of predicted 3s: 85.51724137931035
Average length of predicted 4s: 126.0


In [54]:
def word_count_dict(l):
    word_counts = dict()
    for word in set(l):
        word_counts[word] = l.count(word)
    return dict(sorted(word_counts.items(), key=lambda item: item[1], reverse=True))

In [110]:
wc_dicts = bert_labels.query('pred == 4')['text'].str.lower().str.split().apply(word_count_dict)

word_count_all = dict()
words_all = 0
for wc_dict in wc_dicts:
    for word in wc_dict:
        if word in stop
        if word not in word_count_all.keys():
            word_count_all[word] = wc_dict[word]
        else:
            word_count_all[word] += wc_dict[word]
        words_all += wc_dict[word]

word_count_all = {word: count / words_all for word, count in word_count_all.items()}
            
word_count_all = dict(sorted(word_count_all.items(), key=lambda item: item[1], reverse=True))
list(word_count_all.items())[:50]

[('the', 0.06878306878306878),
 ('and', 0.03527336860670194),
 ('of', 0.031746031746031744),
 ('i', 0.029982363315696647),
 ('a', 0.023809523809523808),
 ('to', 0.01984126984126984),
 ('in', 0.01895943562610229),
 ('is', 0.01763668430335097),
 ('this', 0.015873015873015872),
 ('that', 0.013227513227513227),
 ('it', 0.012786596119929453),
 ('for', 0.010141093474426807),
 ('as', 0.010141093474426807),
 ('was', 0.009700176366843033),
 ('book', 0.009259259259259259),
 ('with', 0.00837742504409171),
 ('but', 0.006613756613756613),
 ('my', 0.006172839506172839),
 ('so', 0.005291005291005291),
 ('or', 0.005291005291005291),
 ('really', 0.005291005291005291),
 ('me', 0.0048500881834215165),
 ('one', 0.0048500881834215165),
 ('like', 0.0048500881834215165),
 ('have', 0.004409171075837742),
 ('much', 0.004409171075837742),
 ('are', 0.004409171075837742),
 ('story', 0.003968253968253968),
 ('up', 0.003968253968253968),
 ('about', 0.003527336860670194),
 ('there', 0.003527336860670194),
 ('charact